# Sentiment Model Prediction Copy and Go

In [1]:
import tensorflow as tf

In [2]:
model = tf.keras.models.load_model("Final_sentiment_predictor")

In [3]:
# Before running this block, you should finish training the DNN with name "model"
import os
import numpy as np  # Remove if already defined

# Predicting
selected_company = ["facebook", "amazon", "tesla", "netflix", "apple", "google"]
company_df = dict(zip(selected_company, [[], [], [], [], [], []]))
for comp in selected_company:
    for month in range(1, 13):
        for day in range(1, 32):
            
            # File reading
            try:
                word_vector = np.load("comment_vectors/{}-{}-{}.npz".format(comp, month, day))
                impacts = list(np.loadtxt("impact_weights/{}-{}-{}.txt".format(comp, month, day)))
            except:
                continue
            
            # Several ways to add weights to the sentiments
            # Plain mean, weighted mean, square weighted mean, sqrt weighted mean
            # All weights normalized before computing
            norm_impacts = [i / (sum(impacts)+0.01) for i in impacts]
            square_impacts = [i**2 for i in impacts]
            norm_square_impacts = [i / (sum(square_impacts)+0.01) for i in square_impacts]
            sqrt_impacts = [i**(1/2) for i in impacts]
            norm_sqrt_impacts = [i / (sum(sqrt_impacts)+0.01) for i in sqrt_impacts]
            
            # Preparing
            date_str = "{}/{}/2020".format(month, day)
            y_preds = model.predict_on_batch(word_vector['inputs'])  # Make predictions
            sentiments_prob = np.array([p[1] for p in y_preds]) * 2 - 1
            sentiments = np.array([p[0] < 0.5 for p in y_preds]).astype(int)  # Manual Softmax
            sentiments = sentiments * 2 - 1  # Change to -1 and 1 labels
            
            # Formatting
            company_df[comp].append([
                date_str,
                np.round(sentiments @ norm_impacts, 3),
                np.round(sentiments @ norm_square_impacts, 3),
                np.round(sentiments @ norm_sqrt_impacts, 3),
                np.round(sentiments_prob @ norm_impacts, 3),
                np.round(sentiments_prob @ norm_square_impacts, 3),
                np.round(sentiments_prob @ norm_sqrt_impacts, 3)
            ])


# Output
import pandas as pd

try:
    os.mkdir("sentiment_time_series")
except FileExistsError:
    print("Directory already exists")

for comp in selected_company:
    df = pd.DataFrame(
        data=company_df[comp],
        columns=["date", "binary_mean", "binary_squared_mean", "binary_sqrt_mean", "prob_mean", "prob_squared_mean", "prob_sqrt_mean"]
    )
    df.to_csv("sentiment_time_series/{}_sentiment_time_series.csv".format(comp), index=False)

print("Done")

C:\Users\maitr\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars


Done
